In [1]:
import pandas as pd
import numpy as np
import random as rand
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [14]:
class SVM():
    """
    Implementation of SMO
    """
    def __init__(self, max_iter=1000, kernel_type='linear', C=1.0, eps=0.001):
        self.max_iter = max_iter
        self.C = C
        self.eps = 0.001
        self.kernel_type = kernel_type
        self.kernel = {
            'linear' : self.linear_kernel,
            'quadratic' : self.quadratic_kernel
        }
        
    
    def get_rand_number(self,low_lim,high_lim,i):
        k = i
        count = 0
        while k == i and count<1000:
            rand_numb = rand.randint(low_lim,high_lim)
            count += 1
        return(rand_numb)
    
    def L_H_calc(self,y_i,y_j,alpha_i,alpha_j,C):
        if y_i==y_j:
            L = max(0,alpha_i+alpha_j-C)
            H = min(C,alpha_i+alpha_j)
        else:
            L = max(0,alpha_j-alpha_i)
            H = min(C,C+alpha_j-alpha_i)
        return(L,H)
    
    def kernel(self,x_i, x_j):
        k = np.dot(x_i,x_j.T)
        return(k)
    
    def func_val(self,x_i,alpha,y,x):
        alpha_into_y = np.multiply(alpha,y)
        w = np.dot(x.T,alpha_into_y)
        b = np.mean(y - np.dot(w.T, x.T))
        f_val = np.sign(np.dot(w.T,x_i) + b).astype(int)
        return(f_val,w,b)
    
    def Err_val(self,f_x,y):
        Err = f_x - y
        return(Err)
    
    def linear_kernel(self,x_i, x_j):
        lin_ker = np.dot(x_i,x_j.T)
        return(lin_ker)
    
    def quadratic_kernel(self,x_i,x_j):
        quad_ker = (np.dot(x_i,x_j.T))**2
        return(quad_ker)
    
    def fit(self,X,y):
        """
        Input Parameters
        ================
        X - Input data
        y - Output data
        
        Output Parameters
        ================
        alpha 
        """
        # Initialization
        n, d = X.shape[0], X.shape[1]
        alpha = np.zeros((n))
        kernel = self.kernel[self.kernel_type]
        count = 0
        
        while count < self.max_iter:
            count += 1
            alpha_old = np.copy(alpha)
            
            for i in range(0,n):
                j = self.get_rand_number(0,n-1,i) # get random number i~=j
                
                x_i,y_i = X[i,:], y[i]
                x_j,y_j = X[j,:], y[j]
                
                eta = kernel(x_i, x_i) + kernel(x_j, x_j) - 2 * kernel(x_i, x_j)
                if eta ==0:
                    continue
                
                alpha_tmp_i,alpha_tmp_j = alpha[i], alpha[j]
                
                # Bounds for Lagrangian multipliers
                L,H = self.L_H_calc(y_i,y_j,alpha_tmp_i,alpha_tmp_j,self.C)
                
                # Compute function and error
                f_xi,wi,bi = self.func_val(x_i,alpha,y,X)
            
                Err_i = self.Err_val(f_xi,y_i)
                f_xj,wj,bj = self.func_val(x_j,alpha,y,X)
                Err_j = self.Err_val(f_xj,y_j)
                
                # Update values of alpha
                alpha[j] = alpha_tmp_j + float(y_j * (Err_i - Err_j))/eta
                alpha[j] = max(alpha[j], L)
                alpha[j] = min(alpha[j], H)
                
                alpha[i] = alpha_tmp_i+(y[i]*y[j]*(alpha_tmp_j-alpha[j]))
            
            
            alpha_diff = np.linalg.norm(alpha-alpha_old)
            print('Iteration -',count)
            print('alpha_difference -',alpha_diff)
            
            if alpha_diff < self.eps:
                print("Model reached convergence criteria")
                break
                
            if count >= self.max_iter:
                print("Max iterations reached")
                break
            
        # Identify support vectors
        alpha_idx = np.where(alpha>0)
        support_vectors = X[alpha_idx,:]
        
        f_x,w,b = self.func_val(X[1,:],alpha,y,X)
        
        return support_vectors,w,b

In [15]:
# Generate synthetic data - Binary class
X,y = make_classification(n_samples=1000, n_features=2, n_informative=2, n_redundant=0, n_classes=2)

# Test-Train split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(X_train.shape,y_train.shape)
print(np.unique(y_train, return_counts=True))

zero_id = np.where(y_train==0)
y_train[zero_id] = -1

zero_id = np.where(y_test==0)
y_test[zero_id] = -1

print(np.unique(y_train, return_counts=True))

(700, 2) (700,)
(array([0, 1]), array([355, 345], dtype=int64))
(array([-1,  1]), array([355, 345], dtype=int64))


In [16]:
model = SVM(max_iter=100, kernel_type='linear', C=1.0, eps=0.01)
support_vectors,w,b = model.fit(X_train,y_train)

Iteration - 1
alpha_difference - 7.037813126037299
Iteration - 2
alpha_difference - 4.801460706988365
Iteration - 3
alpha_difference - 3.0490879913757722
Iteration - 4
alpha_difference - 3.204743844441131
Iteration - 5
alpha_difference - 1.7692439765654142
Iteration - 6
alpha_difference - 0.718330477263191
Iteration - 7
alpha_difference - 1.075629700334484
Iteration - 8
alpha_difference - 0.24287641379866004
Iteration - 9
alpha_difference - 0.75602304321379
Iteration - 10
alpha_difference - 0.0956720107919485
Iteration - 11
alpha_difference - 0.18486077754446523
Iteration - 12
alpha_difference - 0.09091158197653922
Iteration - 13
alpha_difference - 2.830524433501838e-16
Model reached convergence criteria


In [23]:
y_hat = np.sign(np.dot(w.T, X_test.T) + b).astype(int)

In [24]:
confusion_matrix(y_test,y_hat)

array([[130,  18],
       [ 13, 139]], dtype=int64)

In [25]:
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

# metrics
confusion_matrix(y_test,y_pred)

array([[132,  16],
       [ 12, 140]], dtype=int64)